In [1]:
import pandas as pd
import numpy as np
import os,sys
import json
import matplotlib.pyplot as plt
import matplotlib
from sklearn.metrics.pairwise import cosine_similarity
import seaborn as sns

In [2]:
train_df = pd.read_csv('/Users/Administrator/Downloads/FakeNewsWSDM/train.csv')
test_df = pd.read_csv('/Users/Administrator/Downloads/FakeNewsWSDM/test.csv') 

In [3]:
temp_df = train_df.ix[(train_df['label'] == 'disagreed'),['id', 'tid1','tid2', 'title1_zh', 'title2_zh','title1_en','title2_en','label']]

for i in range(4):
    temp_df = temp_df.append(temp_df,ignore_index=True)
    
temp_df_agreed = train_df.ix[(train_df['label'] == 'agreed'),['id', 'tid1','tid2', 'title1_zh', 'title2_zh','title1_en','title2_en','label']]
    
train_df = train_df.append(temp_df, ignore_index=True)
train_df = train_df.append(temp_df_agreed, ignore_index=True)

/Users/Administrator/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


In [4]:
len(train_df)

545781

In [5]:
data_text_t1_en = train_df[['title1_en']] #+ test_df[['title1_en']]
data_text_t1_en['index'] = train_df.index
documents_t1 = data_text_t1_en

/Users/Administrator/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [6]:
data_text_t2_en = train_df[['title2_en']] #+ test_df[['title2_en']]
data_text_t2_en['index'] = train_df.index
documents_t2 = data_text_t2_en

/Users/Administrator/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [7]:
documents_t2.columns = ['title1_en', 'index']

In [8]:
docs = documents_t1.append(documents_t2, ignore_index=True)

In [10]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')
stemmer = SnowballStemmer('english')

def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/Administrator/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [11]:
processed_docs_merged = docs['title1_en'].map(preprocess)

t1_list = list(processed_docs_merged[:545781])
t2_list = list(processed_docs_merged[545781:])

t1_list_str = ['']*len(t1_list)
t2_list_str = ['']*len(t2_list)
for i in range(len(t1_list)):
    t1_list_str[i] = ' '.join(t1_list[i])
for i in range(len(t2_list)):
    t2_list_str[i] = ' '.join(t2_list[i])

In [19]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
s = 0
count1 = 0
count2 = 0 
count3 = 0

sim_agreed = [0]*len(train_df)
sim_disagreed = [0]*len(train_df)
sim_unrelated = [0]*len(train_df)

for i in range(len(train_df)):
    if train_df['label'][i]=='agreed' and t1_list_str[i] != '' and t2_list_str[i] != '' :
        list = ['']*2
        list[0] = t1_list_str[i]
        list[1] = t2_list_str[i]  
        m = vect.fit_transform(list).toarray()
        sim_agreed[count1] = sum(np.multiply(m[0],m[1]))/len(m[0])
        #s = s + sum(np.multiply(m[0],m[1]))/len(m[0])
        count1 = count1+1
        
    elif train_df['label'][i]=='disagreed' and t1_list_str[i] != '' and t2_list_str[i] != '' :
        list = ['']*2
        list[0] = t1_list_str[i]
        list[1] = t2_list_str[i]  
        m = vect.fit_transform(list).toarray()
        sim_disagreed[count2] = sum(np.multiply(m[0],m[1]))/len(m[0])
        #s = s + sum(np.multiply(m[0],m[1]))/len(m[0])
        count2 = count2+1
        
    elif train_df['label'][i]=='unrelated' and t1_list_str[i] != '' and t2_list_str[i] != '' :
        list = ['']*2
        list[0] = t1_list_str[i]
        list[1] = t2_list_str[i]  
        m = vect.fit_transform(list).toarray()
        sim_unrelated[count3] = sum(np.multiply(m[0],m[1]))/len(m[0])
        #s = s + sum(np.multiply(m[0],m[1]))/len(m[0])
        count3 = count3+1
    
        
sim_agreed = sim_agreed[:count1]
sim_disagreed = sim_disagreed[:count2]
sim_unrelated = sim_unrelated[:count3]

In [20]:
m_list = ['fake', 'deni', 'disclos', 'discourag', 
          'disinform', 'dismiss', 'dispel', 'fals', 'fool', 'myth', 'refut', 'report',
           'respond', 'rumour', 'spread', 'truth']

m1_list = [5, 2, 0.3, 0.3,2,1.5,0.3,2,1,5,3,0.3,0.3,5,0.3,0.3]

count1 = 0
count2 = 0 
count3 = 0

sim_agreed_m1 = [0]*len(train_df)
sim_disagreed_m1 = [0]*len(train_df)
sim_unrelated_m1 = [0]*len(train_df)

for i in range(len(train_df)):
    if train_df['label'][i]=='agreed' and t1_list_str[i] != '' and t2_list_str[i] != '' :
        list1 = [0] * 16
        list2 = [0] *16
        for j in range(len(m_list)):
            if(m_list[j] in t1_list_str[i]):
                list1[j] = 1
        for j in range(len(m_list)):
            if(m_list[j] in t2_list_str[i]):
                list2[j] = 1
        sim_agreed_m1[count1] = sum(np.multiply(m1_list,list2)) - sum(np.multiply(m1_list,list1)) /len(m1_list)
        #s = s + sum(np.multiply(m[0],m[1]))/len(m[0])
        count1 = count1+1
        
    elif train_df['label'][i]=='disagreed' and t1_list_str[i] != '' and t2_list_str[i] != '' :
        list1 = [0] * 16
        list2 = [0] *16
        for j in range(len(m_list)):
            if(m_list[j] in t1_list_str[i]):
                list1[j] = 1
        for j in range(len(m_list)):
            if(m_list[j] in t2_list_str[i]):
                list2[j] = 1
        sim_disagreed_m1[count2] = sum(np.multiply(m1_list,list2)) - sum(np.multiply(m1_list,list1)) /len(m1_list)
        #s = s + sum(np.multiply(m[0],m[1]))/len(m[0])
        count2 = count2+1
        
    elif train_df['label'][i]=='unrelated' and t1_list_str[i] != '' and t2_list_str[i] != '' :
        list1 = [0] * 16
        list2 = [0] *16
        for j in range(len(m_list)):
            if(m_list[j] in t1_list_str[i]):
                list1[j] = 1
        for j in range(len(m_list)):
            if(m_list[j] in t2_list_str[i]):
                list2[j] = 1
        sim_unrelated_m1[count3] = sum(np.multiply(m1_list,list2)) - sum(np.multiply(m1_list,list1)) /len(m1_list)
        #s = s + sum(np.multiply(m[0],m[1]))/len(m[0])
        count3 = count3+1
    
        
sim_agreed_m1 = sim_agreed_m1[:count1]
sim_disagreed_m1 = sim_disagreed_m1[:count2]
sim_unrelated_m1 = sim_unrelated_m1[:count3]

In [1]:
sim_agreed_m1

NameError: name 'sim_agreed_m1' is not defined

In [23]:
l = len(sim_unrelated_m1) + len(sim_agreed_m1) + len(sim_disagreed_m1)
X =[0] * l
for i in range(l):
    X[i] = [0] *3
y =[0]*l

In [24]:
count1 = 0
count2 = 0 
count3 = 0
countj = 0

l = len(sim_unrelated_m1) + len(sim_agreed_m1) + len(sim_disagreed_m1)



for i in range(len(train_df)):
    if train_df['label'][i]=='agreed' and t1_list_str[i] != '' and t2_list_str[i] != '' :
        X[countj][0] = sim_agreed[count1] 
        X[countj][1] = sim_agreed_m1[count1]
        X[countj][2] = sim[countj]
        y[countj] = 1
        count1 = count1+1
        countj = countj+1
        
    elif train_df['label'][i]=='disagreed' and t1_list_str[i] != '' and t2_list_str[i] != '' :
        X[countj][0] = sim_disagreed[count2] 
        X[countj][1] = sim_disagreed_m1[count2]
        X[countj][2] = sim[countj]
        y[countj] = 2
        count2 = count2+1
        countj = countj+1
        
    elif train_df['label'][i]=='unrelated' and t1_list_str[i] != '' and t2_list_str[i] != '' :
        X[countj][0] = sim_unrelated[count3] 
        X[countj][1] = sim_unrelated_m1[count3]
        X[countj][2] = sim[countj]
        y[countj] = 0
        count3 = count3+1
        countj = countj+1
        
        
        
        

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline


vectorizer = TfidfVectorizer(stop_words='english', 
                             use_idf=True, 
                             smooth_idf=True)

svd_model = TruncatedSVD(n_components=100,         
                         algorithm='randomized',
                         n_iter=10)

svd_transformer = Pipeline([('tfidf', vectorizer), 
                            ('svd', svd_model)])

svd_matrix = svd_transformer.fit_transform(docs['title1_en'])

In [13]:
svd_matrix[545781:,:].shape

(545781, 100)

In [14]:
svd_matrix_t1 = svd_matrix[:545781,:]

In [15]:
svd_matrix_t2 = svd_matrix[545781:,:]

In [ ]:

#sim = cosine_similarity(svd_matrix_t1, svd_matrix_t2, dense_output=True)

In [ ]:
svd_matrix_t1[0,:]

In [ ]:
len(svd_matrix_t1)

In [16]:
sim = [0] * len(svd_matrix_t1)
for i in range(len(svd_matrix_t1)):
    sim[i] = cosine_similarity(svd_matrix_t1[i,:].reshape(1,-1), svd_matrix_t2[i,:].reshape(1,-1))

In [17]:
for i in range(len(sim)):
    sim[i] = sim[i][0][0]

In [ ]:
 y = train_df.loc[:,'label']

In [ ]:
y_new =[0] * len(y)
for i in range(len(y)):
    if y[i] == 'unrelated':
        y_new[i] = 0
    elif y[i] == 'agreed':
        y_new[i] = 1
    elif y[i] == 'disagreed':
        y_new[i] = 2

In [ ]:
#y_new

In [ ]:
#x_new = list(zip(sim, y_new))

In [ ]:
#type(x_new[0])

In [ ]:
#from numpy.random import randn
#randn(5,4)

In [ ]:
#x_df = pd.DataFrame(x_new)

In [ ]:
#x_df

In [ ]:
#sns.pairplot(x_df, hue='label')

In [ ]:
#x_df.columns = ['sim', 'label']

In [ ]:
#x_df.groupby('label').describe()

In [ ]:
X = [0] * len(sim)
for i in range(len(sim)):
    X[i] = [sim[i],sim[i]]

In [ ]:
len(X)

In [29]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

# Feature Scaling
#from sklearn.preprocessing import StandardScaler
#sc = StandardScaler()
#X_train = sc.fit_transform(X_train)
#X_test = sc.transform(X_test)




# Fitting Logistic Regression to the Training set
#from sklearn.linear_model import LogisticRegression
#classifier = LogisticRegression(random_state = None, multi_class = 'multinomial', class_weight = 'balanced', solver = 'newton-cg')
#classifier.fit(X_train, y_train)

from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)




In [30]:
cm

array([[25178, 10364,  8240],
       [10195, 20367,  6583],
       [ 7138,  6335, 14665]])

In [31]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.59      0.58      0.58     43782
          1       0.55      0.55      0.55     37145
          2       0.50      0.52      0.51     28138

avg / total       0.55      0.55      0.55    109065



In [27]:
cm

array([[29730,  9511,  4541],
       [ 9382, 27317,   446],
       [ 9210,  9574,  9354]])

In [28]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.62      0.68      0.65     43782
          1       0.59      0.74      0.65     37145
          2       0.65      0.33      0.44     28138

avg / total       0.62      0.61      0.60    109065



In [ ]:
cm

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

In [ ]:
len(test_df)

In [ ]:
from numpy.random import randn
randn()

In [ ]:
test_df

In [ ]:
data_text_t1_en_test = test_df[['title1_en']] #+ test_df[['title1_en']]
data_text_t1_en_test['index'] = test_df.index
documents_t1_test = data_text_t1_en_test

data_text_t2_en_test = test_df[['title2_en']] #+ test_df[['title2_en']]
data_text_t2_en_test['index'] = test_df.index
documents_t2_test = data_text_t2_en_test

documents_t2_test.columns = ['title1_en', 'index']

docs_test = documents_t1_test.append(documents_t2_test, ignore_index=True)

svd_matrix_test = svd_transformer.fit_transform(docs_test['title1_en'])

svd_matrix_t1_test = svd_matrix[:80126,:]

svd_matrix_t2_test = svd_matrix[80126:,:]

sim_test = [0] * len(svd_matrix_t1_test)
for i in range(len(svd_matrix_t1_test)):
    sim_test[i] = cosine_similarity(svd_matrix_t1_test[i,:].reshape(1,-1), svd_matrix_t2_test[i,:].reshape(1,-1))
    
for i in range(len(sim_test)):
    sim_test[i] = sim_test[i][0][0]

y_pred_test = [0]*len(sim_test)

list_of_disagree_words = ["fake", "rumour", "rumor", "myth", "refute", 
                          "refut", "truth is", "wrong", "false", "gossip", "made up"]
for i in range(len(sim_test)):
        
        if sim_test[i] < 0.5:
            y_pred_test[i] = 0
            
        elif any(word in (test_df['title1_en'][i]).lower() for word in list_of_disagree_words) and any(word in (test_df['title2_en'][i]).lower() for word in list_of_disagree_words):
                y_pred_test[i] = 2
        
        else:
            y_pred_test[i] = 1
            
            
                
        
            
        
    
    

In [ ]:
len(y_pred_test)

In [ ]:
j=0
for i in range(len(y_pred_test)):
    if y_pred_test[i] == 2:
        j = j+1
    


In [ ]:
j

In [ ]:
y_pred_final = [0] * len(y_pred_test)
for i in range(len(y_pred_test)):
    if y_pred_test[i] == 0:
        y_pred_final[i] ='unrelated'
    elif y_pred_test[i] == 1:
        y_pred_final[i] = 'agreed'
    elif y_pred_test[i] == 2:
        y_pred_final[i] = 'disagreed'


y_pred_sub = [0] * len(y_pred_test)
for i in range(len(y_pred_test)):
    y_pred_sub[i] = [0] * 2
    
for i in range(len(y_pred_sub)):
    y_pred_sub[i][0] = test_df.iloc[i,0]
    
for i in range(len(y_pred_sub)):
    y_pred_sub[i][1] = y_pred_final[i]
    
out = pd.DataFrame(y_pred_sub, columns = ["Id", "Category"])

out.to_csv('/Users/Administrator/Downloads/FakeNewsWSDM/output7.csv', index = False)

In [ ]:
out

In [ ]:
from matplotlib.colors import ListedColormap
X_set, y_set = X_test, y_test
X1, X2 = np.meshgrid(np.arange(start = X_set[:, 0].min() - 1, stop = X_set[:, 0].max() + 1, step = 0.01),
                     np.arange(start = X_set[:, 1].min() - 1, stop = X_set[:, 1].max() + 1, step = 0.01))
plt.contourf(X1, X2, classifier.predict(np.array([X1.ravel(), X2.ravel()]).T).reshape(X1.shape),
             alpha = 0.75, cmap = ListedColormap(('red', 'green')))
plt.xlim(X1.min(), X1.max())
plt.ylim(X2.min(), X2.max())
for i, j in enumerate(np.unique(y_set)):
    plt.scatter(X_set[y_set == j, 0], X_set[y_set == j, 1],
                c = ListedColormap(('red', 'green'))(i), label = j)
plt.title('Logistic Regression (Test set)')
plt.xlabel('Age')
plt.ylabel('Estimated Salary')
plt.legend()
plt.show()

In [ ]:
(test_df['title1_en'][1]).lower()

In [ ]:
list_of_disagree_words

In [ ]:

for i in range(len(sim_test)):
        if any(word in (test_df['title1_en'][i]).lower() for word in list_of_disagree_words) or any(word in (test_df['title2_en'][i]).lower() for word in list_of_disagree_words):
            print(test_df['title1_en'][i] + '\t')
            print(test_df['title2_en'][i] + '\n')
            print(sim_test[i])

In [ ]:
#np.mean((sim_test))

In [ ]:
#np.mean((sim))

In [ ]:
for i in range(len(sim_test)):
        if sim_test[i] == np.max(sim_test):
            print(test_df['title1_en'][i] + '\t')
            print(test_df['title2_en'][i] + '\n')
            print(sim_test[i])